In [12]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree



# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder



from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm


# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import os 
import sys
sys.path.append("../")
from src import soporte_ajuste as sa

pd.options.display.float_format = "{:,.2f}".format 
pd.set_option("display.max_columns", None)

In [5]:
df=pd.read_csv("datos/datos_coding.csv", index_col=0).reset_index(drop=True)

In [6]:
#separamos x e y para predecir nuestro modelo
x= df.drop(columns="price")
y= df[["price"]]

In [8]:
#Separamos train y test
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size= 0.7, random_state=42)

Preprocesamiento.

In [14]:
# Separamos los datos segun el tipo de variable
categorical_columns = x.select_dtypes(include='object').columns
numerical_columns = x.select_dtypes(include=np.number).columns

# Hacemos la gestion de las variables categoricas
target_encoder = TargetEncoder(cols=categorical_columns)
X_encoded = target_encoder.fit_transform(x, y)

# Escalamos las variables
scaler = StandardScaler()
X_encoded[numerical_columns] = scaler.fit_transform(X_encoded[numerical_columns])

# Guardar los transformadores (TargetEncoder y StandardScaler)
with open('target_encoder.pkl', 'wb') as f:
    pickle.dump(target_encoder, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

Modelo

In [18]:
# Supongamos que X_encoded y y están definidos
# Si y es un DataFrame, conviértelo a Serie
if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]  # Convertir a Serie

# Dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.8, random_state=42)

# Verificar las formas de X_train, X_test, y_train, y_test
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de y_test: {y_test.shape}")

# Entrenamos los datos
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predecimos
y_pred = model.predict(X_test)

# Calculamos el RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Precio real vs Predicción:")
display(pd.DataFrame({'Real': y_test, 'Predicción': y_pred}).head(2))
print(f"\nRMSE: {rmse:.2f}")


Forma de X_train: (70, 24)
Forma de X_test: (283, 24)
Forma de y_train: (70,)
Forma de y_test: (283,)
Precio real vs Predicción:


,Real,Predicción
219,720.00,720.02
42,699.00,699.12



RMSE: 12.44
